## Notebook 4: TTS Workflow

We have the exact podcast transcripts ready now. 

Credit: [This](https://colab.research.google.com/drive/1dWWkZzvu7L9Bunq9zvD-W02RFUXoW-Pd?usp=sharing#scrollTo=68QtoUqPWdLk) Colab was used for starter code


In [4]:
#!pip3 install optimum
#!pip install -U flash-attn --no-build-isolation

In [1]:
from IPython.display import Audio
import IPython.display as ipd
from tqdm import tqdm

In [2]:
from transformers import BarkModel, AutoProcessor, AutoTokenizer
import torch
import json
import numpy as np
from parler_tts import ParlerTTSForConditionalGeneration

Flash attention 2 is not installed


### Testing the workflow

In [3]:
device = "cuda:3"

processor = AutoProcessor.from_pretrained("suno/bark")

#model =  model.to_bettertransformer()
#model = BarkModel.from_pretrained("suno/bark", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)
model = BarkModel.from_pretrained("suno/bark", torch_dtype=torch.float16).to(device)#.to_bettertransformer()

Flash attention 2 is not installed
/home/sanyambhutani/.conda/envs/final-checking-meta/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/sanyambhutani/.conda/envs/final-checking-meta/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [3]:
# Set up device
device = "cuda:4" if torch.cuda.is_available() else "cpu"

# Load model and tokenizer
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")

# Define text and description
text_prompt = ""
description = """
Laura's voice is expressive and dramatic in delivery, speaking at a fast pace with a very close recording that almost has no background noise.
"""
# Tokenize inputs
input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(text_prompt, return_tensors="pt").input_ids.to(device)

# Generate audio
generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()

# Play audio in notebook
ipd.Audio(audio_arr, rate=model.config.sampling_rate)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
voice_preset = "v2/en_speaker_6"
sampling_rate = 24000

In [ ]:
text_prompt = """
Exactly! [sigh] And the distillation part is where you take a LARGE-model,and compress-it down into a smaller, more efficient model that can run on devices with limited resources.
"""
inputs = processor(text_prompt, voice_preset=voice_preset).to(device)

speech_output = model.generate(**inputs, temperature = 0.9, semantic_temperature = 0.8)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
text_prompt = """
Exactly! [gasps] And the distillation part is where you take a LARGE-model,and compress-it down into a smaller, more efficient model that can run on devices with limited resources.
"""
inputs = processor(text_prompt, voice_preset=voice_preset).to(device)

speech_output = model.generate(**inputs, temperature = 0.9, semantic_temperature = 0.8)
Audio(speech_output[0].cpu().numpy(), rate=sampling_rate)

###Example extract workflow testing

In [4]:
PODCAST_TEXT = [
    ("Speaker 1", "Welcome to our podcast, where we explore the latest advancements in AI and technology. I'm your host, and today we're joined by a renowned expert in the field of AI. We're going to dive into the exciting world of Llama 3.2, the latest release from Meta AI."),
    ("Speaker 2", "Hi, I'm excited to be here! So, what is Llama 3.2?"),
    ("Speaker 1", "Ah, great question! Llama 3.2 is an open-source AI model that allows developers to fine-tune, distill, and deploy AI models anywhere. It's a significant update from the previous version, with improved performance, efficiency, and customization options."),
    ("Speaker 2", "That sounds amazing! What are some of the key features of Llama 3.2?"),
    ("Speaker 1", "Well, one of the major updates is the introduction of multimodal models that can handle both text and image inputs. This opens up a wide range of applications, from image understanding to visual reasoning."),
    ("Speaker 2", "Hmm, I'm intrigued. Can you give me an example of how this could be used in real life?"),
    ("Speaker 1", "Sure thing! Imagine you're developing an AI-powered virtual assistant that can understand and respond to voice commands, as well as recognize and interact with objects in the physical world."),
    ("Speaker 2", "Wow, that sounds like science fiction! But what about the technical details? How does Llama 3.2 achieve this level of performance?"),
    ("Speaker 1", "Ah, great question! Llama 3.2 uses a combination of techniques, including instruction-tuned benchmarks, vision instruction-tuned benchmarks, and category-specific benchmarks."),
    ("Speaker 2", "Okay, let's dive deeper into the technical details. Can you explain how the instruction-tuned benchmarks work?"),
    ("Speaker 1", "Sure thing! The instruction-tuned benchmarks are designed to evaluate the model's ability to follow instructions and complete tasks. This is done by fine-tuning the model on a specific task, such as language translation or question-answering."),
    ("Speaker 2", "I see. And what about the vision instruction-tuned benchmarks?"),
    ("Speaker 1", "Ah, those are designed to evaluate the model's ability to understand and interact with visual data. This includes tasks such as image classification, object detection, and visual reasoning."),
    ("Speaker 2", "Okay, got it. And what about the category-specific benchmarks?"),
    ("Speaker 1", "Those are designed to evaluate the model's performance on specific tasks or categories, such as math, reasoning, or tool use."),
    ("Speaker 2", "Hmm, I'm curious. Can you show me some examples of how Llama 3.2 performs on these benchmarks?"),
    ("Speaker 1", "Sure thing! Let me pull up some results. (pause) Okay, so on the MMLU benchmark, Llama 3.2 achieves a score of 63.4, outperforming the previous state-of-the-art model."),
    ("Speaker 2", "Wow, those results are impressive! But what about real-world applications? How is Llama 3.2 being used in industry and academia?"),
    ("Speaker 1", "Ah, great question! Llama 3.2 is being used in a wide range of applications, from virtual assistants to medical diagnosis. We're also seeing partnerships with major companies, such as ARM, MediaTek, and Qualcomm."),
    ("Speaker 2", "That's amazing! Can you tell me more about these partnerships?"),
    ("Speaker 1", "Sure thing! These partnerships are enabling the development of edge AI and vision applications, such as smart home devices, autonomous vehicles, and medical imaging."),
    ("Speaker 2", "Hmm, I'm intrigued. Can you give me an example of how Llama 3.2 is being used in one of these applications?"),
    ("Speaker 1", "Sure thing! Let me tell you about a project we're working on with DoorDash. They're using Llama 3.2 to develop an AI-powered chatbot that can understand and respond to customer inquiries."),
    ("Speaker 2", "Wow, that's amazing! I think we've covered a lot of ground today. Can you summarize the key points for our listeners?"),
    ("Speaker 1", "Sure thing! Llama 3.2 is a significant update from the previous version, with improved performance, efficiency, and customization options. We're seeing a wide range of applications, from virtual assistants to medical diagnosis, and partnerships with major companies."),
    ("Speaker 2", "Hmm, I'm excited to see where this technology will take us. Thanks for joining me today!"),
    ("Speaker 1", "Thanks for having me!"),
]

In [ ]:
speaker1_voice = "v2/en_speaker_6"
speaker1_segments = []

In [ ]:
speaker2_voice = "v2/en_speaker_9"
speaker2_segments = []

In [ ]:
generated_segments = []

In [ ]:
for speaker, text in tqdm(conversation, desc="Generating podcast segments", unit="segment"):
    # Pick voice based on speaker
    voice = "v2/en_speaker_6" if speaker == "Speaker 1" else "v2/en_speaker_9"
    
    # Generate
    inputs = processor(text, voice_preset=voice).to(device)
    audio = model.generate(**inputs, temperature=0.9, semantic_temperature=0.8)
    
    # Add to our list
    generated_segments.append(audio[0].cpu().numpy())
    
    # Optional: Play as we go (you might want to comment this out for large conversations)
    display(Audio(audio[0].cpu().numpy(), rate=sampling_rate))

In [ ]:
# Stick it all together at the end
final_podcast = np.concatenate(generated_segments)

In [ ]:
final_podcast_32 = final_podcast.astype(np.float32)

In [ ]:
from scipy.io import wavfile
wavfile.write("podcast.wav", sampling_rate, final_podcast_32)

# Play the whole thing
Audio(final_podcast, rate=sampling_rate)

In [ ]:
from pydub import AudioSegment
import io

In [ ]:
audio_int16 = (final_podcast * 32767).astype(np.int16)

# Convert to AudioSegment
byte_io = io.BytesIO()
wavfile.write(byte_io, sampling_rate, audio_int16)
byte_io.seek(0)
audio_segment = AudioSegment.from_wav(byte_io)

# Export as MP3 with good quality
audio_segment.export("podcast.mp3", 
                    format="mp3", 
                    bitrate="192k",  # Adjust bitrate as needed (128k, 192k, 256k, 320k)
                    parameters=["-q:a", "0"])  # Highest quality

# Play the result
Audio("podcast.mp3")

## Bringing it together

In [3]:
import pickle

with open('./podcast_ready_data.pkl', 'rb') as file:
    PODCAST_TEXT = pickle.load(file)

In [4]:
bark_processor = AutoProcessor.from_pretrained("suno/bark")
bark_model = BarkModel.from_pretrained("suno/bark", torch_dtype=torch.float16).to("cuda:3")
bark_sampling_rate = 24000

/home/sanyambhutani/.conda/envs/final-checking-meta/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/sanyambhutani/.conda/envs/final-checking-meta/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [5]:
parler_model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1").to("cuda:3")
parler_tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")

In [6]:
speaker1_description = """
Laura's voice is expressive and dramatic in delivery, speaking at a moderately fast pace with a very close recording that almost has no background noise.
"""

In [7]:
generated_segments = []
sampling_rates = []  # We'll need to keep track of sampling rates for each segment

In [8]:
device="cuda:3"

In [9]:
def generate_speaker1_audio(text):
    """Generate audio using ParlerTTS for Speaker 1"""
    input_ids = parler_tokenizer(speaker1_description, return_tensors="pt").input_ids.to(device)
    prompt_input_ids = parler_tokenizer(text, return_tensors="pt").input_ids.to(device)
    generation = parler_model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
    audio_arr = generation.cpu().numpy().squeeze()
    return audio_arr, parler_model.config.sampling_rate

In [10]:
def generate_speaker2_audio(text):
    """Generate audio using Bark for Speaker 2"""
    inputs = bark_processor(text, voice_preset="v2/en_speaker_6").to(device)
    speech_output = bark_model.generate(**inputs, temperature=0.9, semantic_temperature=0.8)
    audio_arr = speech_output[0].cpu().numpy()
    return audio_arr, bark_sampling_rate


In [38]:
def numpy_to_audio_segment(audio_arr, sampling_rate):
    """Convert numpy array to AudioSegment"""
    # Convert to 16-bit PCM
    audio_int16 = (audio_arr * 32767).astype(np.int16)
    
    # Create WAV file in memory
    byte_io = io.BytesIO()
    wavfile.write(byte_io, sampling_rate, audio_int16)
    byte_io.seek(0)
    
    # Convert to AudioSegment
    return AudioSegment.from_wav(byte_io)

In [16]:
PODCAST_TEXT

'[\n    ("Speaker 1", "Welcome to this week\'s episode of AI Insights, where we explore the latest developments in the field of artificial intelligence. Today, we\'re going to dive into the fascinating world of knowledge distillation, a methodology that transfers advanced capabilities from leading proprietary Large Language Models, or LLMs, to their open-source counterparts. Joining me on this journey is my co-host, who\'s new to the topic, and I\'ll be guiding them through the ins and outs of knowledge distillation. So, let\'s get started!"),\n    ("Speaker 2", "Sounds exciting! I\'ve heard of knowledge distillation, but I\'m not entirely sure what it\'s all about. Can you give me a brief overview?"),\n    ("Speaker 1", "Of course! Knowledge distillation is a technique that enables the transfer of knowledge from a large, complex model, like GPT-4 or Gemini, to a smaller, more efficient model, like LLaMA or Mistral. This process allows the smaller model to learn from the teacher model\

In [18]:
import ast
ast.literal_eval(PODCAST_TEXT)

[('Speaker 1',
  "Welcome to this week's episode of AI Insights, where we explore the latest developments in the field of artificial intelligence. Today, we're going to dive into the fascinating world of knowledge distillation, a methodology that transfers advanced capabilities from leading proprietary Large Language Models, or LLMs, to their open-source counterparts. Joining me on this journey is my co-host, who's new to the topic, and I'll be guiding them through the ins and outs of knowledge distillation. So, let's get started!"),
 ('Speaker 2',
  "Sounds exciting! I've heard of knowledge distillation, but I'm not entirely sure what it's all about. Can you give me a brief overview?"),
 ('Speaker 1',
  "Of course! Knowledge distillation is a technique that enables the transfer of knowledge from a large, complex model, like GPT-4 or Gemini, to a smaller, more efficient model, like LLaMA or Mistral. This process allows the smaller model to learn from the teacher model's output, enablin

In [39]:
final_audio = None

for speaker, text in tqdm(ast.literal_eval(PODCAST_TEXT), desc="Generating podcast segments", unit="segment"):
    if speaker == "Speaker 1":
        audio_arr, rate = generate_speaker1_audio(text)
    else:  # Speaker 2
        audio_arr, rate = generate_speaker2_audio(text)
    
    # Convert to AudioSegment (pydub will handle sample rate conversion automatically)
    audio_segment = numpy_to_audio_segment(audio_arr, rate)
    
    # Add to final audio
    if final_audio is None:
        final_audio = audio_segment
    else:
        final_audio += audio_segment

Generating podcast segments:   6%|███▉                                                          | 1/16 [00:20<05:02, 20.16s/segment]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
Generating podcast segments:  19%|███████████▋                                                  | 3/16 [01:02<04:33, 21.06s/segment]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
Generating podcast segments:  31%|███████████████████▍                                          | 5/16 [01:41<03:30, 19.18s/segment]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected beh

In [40]:
final_audio.export("_podcast.mp3", 
                  format="mp3", 
                  bitrate="192k",
                  parameters=["-q:a", "0"])

<_io.BufferedRandom name='_podcast.mp3'>

In [ ]:
#fin